In [ ]:
import pickle 
from importlib import reload

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import models.custom.detector as g
from libs.dataset_utils import prepare_data_from_tfrecord
from libs.image_utils import (get_random_patch_list,
                              draw_bounding_box,
                              calc_iou_accuracy,
                              calc_iou_top_1_accuracy)
from libs.various_utils import load_from_pickle
from configs.project_config import project_path

from helpers import (evaluate,
                     visualize,
                     visualize_cam,
                     restore_preprocessed)

In [ ]:
"""
Constant
"""
NUM_CLASS = 200
INPUT_SHAPE = (64, 64, 3)
MODEL_BASE_INPUT_SHAPE = (224, 224, 3)

tfrecord_train_dir = "{}/data/tiny_imagenet_200/tfrecord/train".format(project_path)
tfrecord_valid_dir = "{}/data/tiny_imagenet_200/tfrecord/valid".format(project_path)
tfrecord_test_dir = "{}/data/tiny_imagenet_200/tfrecord/test".format(project_path)

meta_path = "{}/data/tiny_imagenet_200/meta.pickle".format(project_path)
pretrained_ckpt_path = "{}/checkpoints/inception_v3/inception_v3.ckpt".format(project_path)

vanila_model_save_path = "{}/checkpoints/vanila_inception_v3/vanila_inception_v3".format(project_path)

pickle_data_full_path = "{}/data/tiny_imagenet_200/pickle/tiny_imagenet.pickle".format(project_path)

# Build Model, Dataset Pipeline and Load Pretrained Inception Weights

In [ ]:
# ==============================================================================
# model_base_name : { "InceptionV3", "InceptionV3", "alexnet_v2"}
# 각 모델마다 final endpoint를 변경할 수 있다. final endpoint 의 정보는 models/안에 모델별 폴더안의 python 파일에 들어있다.
model_base_name = "InceptionV3"
model = g.Detector(output_dim=NUM_CLASS,
                   input_shape=INPUT_SHAPE,
                   model_base_input_shape=MODEL_BASE_INPUT_SHAPE,
                   model_base_name=model_base_name,
                   model_base_final_endpoint='Mixed_7c',
                   model_name="hide_and_seek")

In [ ]:
"""
Read Data Pickle
"""
try :
    d = load_from_pickle(pickle_data_full_path)
    (X_train, P_train, Y_train, Y_train_one_hot) = (d['X_train'], d['P_train'], d['Y_train'] ,d['Y_train_one_hot'])
    (X_valid, P_valid, Y_valid, Y_valid_one_hot) = (d['X_valid'], d['P_valid'], d['Y_valid'] ,d['Y_valid_one_hot'])
    (X_test, P_test, Y_test, Y_test_one_hot) = (d['X_test'], d['P_test'], d['Y_test'] ,d['Y_test_one_hot'])
    (idx_word_dict, word_idx_dict) = (d['idx_word_dict'], d['word_idx_dict'])
    (idx_nid_dict, nid_idx_dict) = (d['idx_nid_dict'], d['nid_idx_dict'])
except Exception as e: 
    print("maybe there is no pickle data: {}".format(e))

In [ ]:
# ==============================================================================
with model.g.as_default():
    """
    Read Data
    """
    d = prepare_data_from_tfrecord(
        tfrecord_train_dir=tfrecord_train_dir,
        tfrecord_valid_dir=tfrecord_valid_dir,
        tfrecord_test_dir=tfrecord_test_dir,
        batch_size=200,
        shuffle_buffer_size=1000)
    (X, Y, P,
     init_dataset_train,
     init_dataset_train_has,
     init_dataset_valid,
     init_dataset_test,
     
     X_raw, Y_raw, P_raw,
     init_dataset_train_raw,
     init_dataset_valid_raw,
     init_dataset_test_raw,) = (d['X'], d['Y'], d['P'],
                            d['init_dataset_train'],
                            d['init_dataset_train_has'],
                            d['init_dataset_valid'],
                            d['init_dataset_test'],
                            d['X_raw'], d['Y_raw'], d['P_raw'],
                            d['init_dataset_train_raw'],
                            d['init_dataset_valid_raw'],
                            d['init_dataset_test_raw'])

    meta = load_from_pickle(meta_path)
    model.meta.update(meta)

    """
    Initialize with pretrained weights
    """
    variables_to_restore = tf.contrib.framework.get_variables_to_restore(
        include=[model_base_name])
    init_pretrain_fn = tf.contrib.framework.assign_from_checkpoint_fn(
        pretrained_ckpt_path, variables_to_restore)

    init_pretrain_fn(model.sess)

# Vanila Model - Learning Deep Features for Discriminative Localization

In [ ]:
# ==============================================================================

model.train_with_dataset_api(X=X,
                             Y=Y,
                             init_dataset_train=init_dataset_train,
                             init_dataset_valid=init_dataset_valid,
                             n_epoch=2,
                             learning_rate=0.001,
                             reg_lambda=0.,
                             dropout_keep_prob=0.8,
                             patience=10,
                             verbose_interval=1,
                             mode=g.MODE_TRAIN_ONLY_CLF,
                             save_dir_path=None)

In [ ]:
model.train_with_dataset_api(X=X,
                             Y=Y,
                             init_dataset_train=init_dataset_train,
                             init_dataset_valid=init_dataset_valid,
                             n_epoch=1,
                             learning_rate=0.001,
                             reg_lambda=0.,
                             dropout_keep_prob=0.8,
                             patience=10,
                             verbose_interval=1,
                             mode=g.MODE_TRAIN_GLOBAL,
                             save_dir_path=None)

model.save(vanila_model_save_path)

# Run Sample

In [ ]:
# ==============================================================================
num_sample = 100
X_sample =  X_train[:num_sample]
P_sample = P_train[:num_sample]
Y_sample_one_hot = Y_train_one_hot[:num_sample]
Y_sample = Y_train[:num_sample]

In [ ]:
(cam_list_sample,
bbox_list_sample,
gt_known_loc_accuracy_sample,
top_1_loc_accuracy_sample) = evaluate(model, X_sample, P_sample, Y_sample_one_hot, 'sample')

In [ ]:
visualize_cam(X_sample, Y_sample, cam_list_sample, idx_word_dict, n_show=2, start=16)

# Evaluation

In [ ]:
(cam_list_train,
bbox_list_train,
gt_known_loc_accuracy_train,
top_1_loc_accuracy_train) = evaluate(model, X_train, P_train, Y_train_one_hot, 'train',
                                     flag_preprocess=False)

In [ ]:
(cam_list_valid,
bbox_list_valid,
gt_known_loc_accuracy_valid,
top_1_loc_accuracy_valid) = evaluate(model, X_valid, P_valid, Y_valid_one_hot, 'valid',
                                     flag_preprocess=False)

In [ ]:
(cam_list_test,
bbox_list_test,
gt_known_loc_accuracy_test,
top_1_loc_accuracy_test) = evaluate(model, X_test, P_test, Y_test_one_hot, 'test',
                                    flag_preprocess=False)

# Visualization

In [ ]:
visualize(X_train, P_train, Y_train, cam_list_train, bbox_list_train, idx_word_dict,
          n_show=3, start=0)

In [ ]:
visualize(X_valid, P_valid, Y_valid, cam_list_valid, bbox_list_valid, idx_word_dict,
          n_show=3, start=0)

In [ ]:
visualize(X_test, P_test, Y_test, cam_list_test, bbox_list_test, idx_word_dict,
          n_show=3, start=6)